In [133]:
# load nyc_motor_vehicle_collisions dataset
import pandas as pd
df = pd.read_csv('../data/raw/nyc_motor_vehicle_collisions.csv')
df.head()

/var/folders/_6/p9nq2tdd6418xp1vt5dxwqhm0000gn/T/ipykernel_13920/1411559061.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/nyc_motor_vehicle_collisions.csv')


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2021-09-11T00:00:00.000,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26T00:00:00.000,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2023-11-01T00:00:00.000,1:29,BROOKLYN,11230.0,40.62179,-73.970024,"\n, \n(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NaN,...,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN
3,2022-06-29T00:00:00.000,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
4,2022-09-21T00:00:00.000,13:21,NaN,NaN,NaN,NaN,NaN,BROOKLYN BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4566131,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN


In [134]:
# determine which columns have mixed data types, or more than one data type
# Find columns with mixed data types
mixed_type_cols = []
for col in df.columns:
    # Check if the column has more than one data type
    num_types = df[col].apply(type).nunique()
    if num_types > 1:
        mixed_type_cols.append(col)

if mixed_type_cols:
    print("Columns with mixed data types:", mixed_type_cols)
else:
    print("No columns with mixed data types found (besides potentially detected during load).")

Columns with mixed data types: ['borough', 'zip_code', 'location', 'on_street_name', 'off_street_name', 'cross_street_name', 'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2', 'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5', 'vehicle_type_code1', 'vehicle_type_code2', 'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5']


In [135]:
# create duplicate df without any rows with missing data
# then, find columns with mixed data types
df_clone = df.dropna()

mixed_type_cols = []
for col in df_clone.columns:
    # Check if the column has more than one data type
    num_types = df_clone[col].apply(type).nunique()
    if num_types > 1:
        mixed_type_cols.append(col)

if mixed_type_cols:
    print("Columns with mixed data types:", mixed_type_cols)
else:
    print("No columns with mixed data types found (besides potentially detected during load).")

No columns with mixed data types found (besides potentially detected during load).


We've confirmed that NaN values are the sole cause of mixed data types

Create rules-based logic to determine the method of getting rid of rows with missing data.

Rules for following fields:
1. Lat/Lng
2. On Street
3. Off Street
4. Cross Street

1. If a row has lat/lng, keep it
2. If a row has no lat/lng, but it has an off street (address), keep it
3. If a row has no lat/lng, but it has on street AND cross street, keep it
4. If a row has no lat/lng, but it has on street AND nothing else, determine if it contains objective location date (ie bridge, tunnel, etc.), and if so, keep it
5. If a row has no lat/lng, but it has on street AND nothing else, determine if it contains objective location date (ie bridge, tunnel, etc.), and if not, remove it

For rows that remain and DON'T contain lat/lng values, we must geocode them using either the street/cross street or the off street address, borough.

In [136]:
# count number of rows
df.shape

(2171175, 29)

In [137]:
rows_count_before = 2171175

In [138]:
import pandas as pd
import re

# Define the regex pattern to find the exact words/abbreviations BR or TNL
# \b ensures we match them as separate tokens (words/abbreviations)
# (?i) makes the search case-insensitive
pattern = r'(?i)\b(BRG|BRDG)\b'

# Filter rows where 'on_street_name' contains the pattern
# .astype(str) handles potential non-string values (like NaN) gracefully
br_tnl_rows = df[df['on_street_name'].astype(str).str.contains(pattern, regex=True, na=False)]

# Display the first few matching rows (optional)
print(f"Found {len(br_tnl_rows)} rows containing 'BR' or 'TNL' as distinct words/abbreviations.")
br_tnl_rows

# If you want to see the unique street names that matched:
# print("\nUnique matching 'on_street_name' values:")
# print(br_tnl_rows['on_street_name'].unique())

/var/folders/_6/p9nq2tdd6418xp1vt5dxwqhm0000gn/T/ipykernel_13920/1849328648.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  br_tnl_rows = df[df['on_street_name'].astype(str).str.contains(pattern, regex=True, na=False)]


Found 35 rows containing 'BR' or 'TNL' as distinct words/abbreviations.


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
319596,2024-12-21T00:00:00.000,16:00,MANHATTAN,10033.0,40.848120,-73.930885,"\n, \n(40.84812, -73.930885)",WASHINGTON BRG,W 181 ST,NaN,...,Unspecified,NaN,NaN,NaN,4782802,Sedan,Sedan,NaN,NaN,NaN
1013872,2017-08-17T00:00:00.000,14:12,NaN,NaN,NaN,NaN,NaN,Ed Koch Queensboro Brg-Upper Lev,EAST 58 STREET,NaN,...,NaN,NaN,NaN,NaN,3732723,Sedan,NaN,NaN,NaN,NaN
1290508,2016-05-22T00:00:00.000,1:15,NaN,NaN,NaN,NaN,NaN,QUEENS BOROUGH BRDG,NaN,NaN,...,NaN,NaN,NaN,NaN,3445468,Sedan,NaN,NaN,NaN,NaN
1293263,2016-05-15T00:00:00.000,4:35,NaN,NaN,NaN,NaN,NaN,queens boro brdg,NaN,NaN,...,Unsafe Lane Changing,NaN,NaN,NaN,3441798,4 dr sedan,4 dr sedan,NaN,NaN,NaN
1323188,2016-04-04T00:00:00.000,4:16,NaN,NaN,40.754013,-73.947680,"(40.754013, -73.94768)",Ed Koch Queensboro Brg-Lower Lev,NaN,NaN,...,Unspecified,NaN,NaN,NaN,3408746,CASE,Sedan,NaN,NaN,NaN
1388198,2015-12-18T00:00:00.000,8:30,BRONX,10453.0,40.852706,-73.921835,"(40.8527062, -73.921835)",ROBERTO CLEMENTE STATE PARK BRDG,MAC CRACKEN AVENUE,NaN,...,NaN,NaN,NaN,NaN,3356119,UNKNOWN,NaN,NaN,NaN,NaN
1681744,2014-07-31T00:00:00.000,18:20,BRONX,10453,40.852140,-73.920697,"(40.85214, -73.9206971)",ROBERTO CLEMENTE STATE PARK BRDG,CEDAR AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,1001890,SPORT UTILITY / STATION WAGON,UNKNOWN,NaN,NaN,NaN
2107225,2025-01-01T00:00:00.000,2:35,NaN,NaN,40.734314,-73.938020,"\n, \n(40.734314, -73.93802)",JJ BYRNE MEMORIAL BRG,REVIEW AVE,NaN,...,Driver Inattention/Distraction,NaN,NaN,NaN,4783215,Sedan,Sedan,NaN,NaN,NaN
2112035,2025-01-15T00:00:00.000,15:39,MANHATTAN,10033.0,40.848120,-73.930885,"\n, \n(40.84812, -73.930885)",WASHINGTON BRG,AMSTERDAM AVE,NaN,...,Unspecified,NaN,NaN,NaN,4786103,Sedan,Sedan,NaN,NaN,NaN
2114550,2025-01-22T00:00:00.000,15:13,NaN,NaN,40.743217,-73.951340,"\n, \n(40.743217, -73.95134)",PULASKI BRG,JACKSON AVE,NaN,...,Unspecified,NaN,NaN,NaN,4787720,Sedan,Sedan,NaN,NaN,NaN


In [139]:
# Define the regex pattern to find the exact words/abbreviations BRG, BRDG, or BR
# \b ensures we match them as separate words/abbreviations
pattern = r'\b(BRG|BRDG|BR|br|Br|brg|brdg)\b'

# Replace these occurrences with 'BRIDGE', case-insensitive
# Apply to the 'on_street_name' column
# Note: This modifies the DataFrame 'df' in place
df['on_street_name'] = df['on_street_name'].str.replace(pattern, 'BRIDGE', regex=True, flags=re.IGNORECASE)

# Optional: Display some examples or check unique values to verify
print("Replacement complete. Checking some unique values containing 'BRIDGE':")
print(df[df['on_street_name'].str.contains('BRIDGE', na=False)]['on_street_name'].unique()[:10]) # Show first 10 unique examples

Replacement complete. Checking some unique values containing 'BRIDGE':
['QUEENSBORO BRIDGE UPPER' 'THROGS NECK BRIDGE' 'BROOKLYN BRIDGE'
 'WEST KINGSBRIDGE ROAD' 'WILLIAMSBURG BRIDGE OUTER ROADWA'
 'VERRAZANO BRIDGE' 'VERRAZANO BRIDGE UPPER' 'VERRAZANO BRIDGE LOWER'
 'BRONX WHITESTONE BRIDGE' 'TRIBOROUGH BRIDGE']


In [140]:
# return number of rows where 'on_street_name' contains 'BRIDGE'
df[df['on_street_name'].str.contains('BRIDGE', na=False)].shape

(24485, 29)

In [141]:
# Define the regex pattern to find the exact words/abbreviations BRG, BRDG, or BR
# \b ensures we match them as separate words/abbreviations
pattern = r'\b(tunnel|Tunnel|TNL|tnl|Tnl|TUNNEL)\b'

# Replace these occurrences with 'BRIDGE', case-insensitive
# Apply to the 'on_street_name' column
# Note: This modifies the DataFrame 'df' in place
df['on_street_name'] = df['on_street_name'].str.replace(pattern, 'TUNNEL', regex=True, flags=re.IGNORECASE)

# Optional: Display some examples or check unique values to verify
print("Replacement complete. Checking some unique values containing 'TUNNEL':")
print(df[df['on_street_name'].str.contains('TUNNEL', na=False)]['on_street_name'].unique()[:10]) # Show first 10 unique examples

Replacement complete. Checking some unique values containing 'TUNNEL':
['BROOKLYN BATTERY TUNNEL' 'QUEENS MIDTOWN TUNNEL' 'TUNNEL Exit street'
 'TUNNEL exit street' 'TUNNEL EXIT ST' 'HUGH L. CAREY TUNNEL'
 'TUNNEL EXIT STREET' 'TUNNEL Exit Street' 'TUNNEL exit marginal street'
 'TUNNEL ENTRANCE STREET']


In [142]:
import pandas as pd
import numpy as np # Often needed for NaN checks, though pandas handles it
import re

# Store original row count
rows_before = len(df)

# Condition 1: Latitude is not NaN
cond1 = df['latitude'].notna()

# Condition 2: 'on_street_name' contains BRIDGE or TUNNEL (whole words, case-insensitive)
bridge_tunnel_pattern = r'(?i)\b(BRIDGE|TUNNEL)\b'
# Ensure we handle NaN in 'on_street_name' and use string methods
cond2 = df['on_street_name'].astype(str).str.contains(bridge_tunnel_pattern, regex=True, na=False)

# Condition 3: Both 'on_street_name' AND 'cross_street_name' are not NaN
cond3 = df['on_street_name'].notna() & df['cross_street_name'].notna()

# Condition 4: 'off_street_name' is not NaN
cond4 = df['off_street_name'].notna()

# Combine conditions: Keep rows where AT LEAST ONE condition is met
keep_mask = cond1 | cond2 | cond3 | cond4

# Apply the filter
df_filtered = df[keep_mask].copy() # Create a filtered copy

# --- Alternatively, to modify df in place (use only one of these) ---
# df = df[keep_mask]
# --- End alternative ---

# Report results
rows_after = len(df_filtered) # Or len(df) if modifying in place
print(f"Original rows: {rows_before}")
print(f"Rows after filtering: {rows_after}")
print(f"Rows removed: {rows_before - rows_after}")

# Optional: Display head of the filtered DataFrame
print("\nFiltered DataFrame head:")
df_filtered.head()

# IMPORTANT: If you want to continue working with the filtered data,
# make sure to use 'df_filtered' going forward, or if you modified
# 'df' in place, just continue using 'df'.
# Example: df = df_filtered

/var/folders/_6/p9nq2tdd6418xp1vt5dxwqhm0000gn/T/ipykernel_13920/1858696262.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cond2 = df['on_street_name'].astype(str).str.contains(bridge_tunnel_pattern, regex=True, na=False)


Original rows: 2171175
Rows after filtering: 2087237
Rows removed: 83938

Filtered DataFrame head:


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2021-09-11T00:00:00.000,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26T00:00:00.000,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2023-11-01T00:00:00.000,1:29,BROOKLYN,11230.0,40.62179,-73.970024,"\n, \n(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NaN,...,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN
3,2022-06-29T00:00:00.000,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
4,2022-09-21T00:00:00.000,13:21,NaN,NaN,NaN,NaN,NaN,BROOKLYN BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4566131,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN


In [143]:
df_filtered.shape

(2087237, 29)

In [144]:
new_rows_count = 2087237

print(f"Rows removed: {rows_count_before - new_rows_count}")

Rows removed: 83938


In [145]:
# dates are currently in format 2022-06-29T00:00:00.000, change to 2022-06-29
df_filtered['crash_date'] = df_filtered['crash_date'].str[:10]
df_filtered.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2021-09-11,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2023-11-01,1:29,BROOKLYN,11230.0,40.62179,-73.970024,"\n, \n(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NaN,...,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN
3,2022-06-29,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
4,2022-09-21,13:21,NaN,NaN,NaN,NaN,NaN,BROOKLYN BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4566131,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN


In [146]:
# count of NaN values in each column
df_filtered.isna().sum()

crash_date                             0
crash_time                             0
borough                           593563
zip_code                          593813
latitude                          155994
longitude                         155994
location                          155994
on_street_name                    412104
off_street_name                   744611
cross_street_name                1737455
number_of_persons_injured             16
number_of_persons_killed              29
number_of_pedestrians_injured          0
number_of_pedestrians_killed           0
number_of_cyclist_injured              0
number_of_cyclist_killed               0
number_of_motorist_injured             0
number_of_motorist_killed              0
contributing_factor_vehicle_1       7285
contributing_factor_vehicle_2     332179
contributing_factor_vehicle_3    1938123
contributing_factor_vehicle_4    2053151
contributing_factor_vehicle_5    2077864
collision_id                           0
vehicle_type_cod

In [147]:
df_filtered.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2021-09-11,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2023-11-01,1:29,BROOKLYN,11230.0,40.62179,-73.970024,"\n, \n(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NaN,...,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN
3,2022-06-29,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
4,2022-09-21,13:21,NaN,NaN,NaN,NaN,NaN,BROOKLYN BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4566131,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN


In [148]:
import pandas as pd
import numpy as np # For selecting numeric types

# --- Ensure 'crash_date' is datetime and extract year ---
# Make a copy to avoid SettingWithCopyWarning if df is a slice
df_agg = df.copy()
# Convert 'crash_date' to datetime objects, coercing errors to NaT (Not a Time)
df_agg['crash_date'] = pd.to_datetime(df_agg['crash_date'], errors='coerce')
# Drop rows where conversion failed (optional, but good practice)
df_agg.dropna(subset=['crash_date'], inplace=True)
# Extract the year
df_agg['year'] = df_agg['crash_date'].dt.year
# Convert year to integer (optional, for cleaner display)
df_agg['year'] = df_agg['year'].astype(int)

# --- Select only numeric columns for aggregation ---
# Exclude 'year' itself, and potentially other ID-like columns if they are numeric
# (e.g., collision_id, zip_code might be numeric but summing them isn't meaningful)
# Let's select based on dtype and explicitly exclude known non-summable numerics
numeric_cols = df_agg.select_dtypes(include=np.number).columns.tolist()
cols_to_sum = [col for col in numeric_cols if col not in ['year', 'latitude', 'longitude', 'zip_code', 'collision_id']] # Add other non-sensical IDs if needed

# --- Group by year and sum the selected numeric columns ---
yearly_sums = df_agg.groupby('year')[cols_to_sum].sum()

# --- Add the 'Total' row ---
# Calculate column sums of the aggregated results
total_row = yearly_sums.sum()
# Assign a name to the Series for the index label
total_row.name = 'Total'
# Append the total row using pd.concat
yearly_sums_with_total = pd.concat([yearly_sums, pd.DataFrame(total_row).T])


# --- Display the result ---
print("Sum of numeric columns by year (excluding lat/lon, zip, id):")
yearly_sums_with_total

# If you want integer display for counts:
# print(yearly_sums_with_total.astype(int)) # This might fail if there are NaNs or floats

Sum of numeric columns by year (excluding lat/lon, zip, id):


,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed
2012,27370.0,137.0,5902.0,71.0,2200.0,6.0,19262.0,60.0
2013,55211.0,296.0,12015.0,175.0,4073.0,11.0,39122.0,110.0
2014,51128.0,261.0,11030.0,133.0,3992.0,21.0,36095.0,107.0
2015,51354.0,242.0,10078.0,132.0,4282.0,15.0,36993.0,95.0
2016,60293.0,245.0,11088.0,149.0,4978.0,18.0,43989.0,71.0
2017,60658.0,256.0,11157.0,127.0,4892.0,27.0,44608.0,107.0
2018,62001.0,231.0,11132.0,123.0,4732.0,10.0,46114.0,98.0
2019,61324.0,244.0,10552.0,131.0,4985.0,31.0,45786.0,82.0
2020,44696.0,270.0,6695.0,102.0,5595.0,29.0,32406.0,139.0
2021,51784.0,297.0,7501.0,132.0,4961.0,19.0,37190.0,134.0


In [149]:
# filter out all rows with NaN values for lat/lng coordinates for df_filtered
df_filtered = df_filtered.dropna(subset=['latitude', 'longitude'])

# size of the filtered dataset
df_filtered.shape

(1931243, 29)

In [150]:
# size of the previously filtered df_filtered
df_filtered_before = 2087237

df_filtered_now = 1931243

print(f"Rows removed: {df_filtered_before - df_filtered_now}")

Rows removed: 155994


Now we need to find out if the unabridged dataset is statistically similar enough to a dataset where we filter out all rows with NaN values for lat/lng coordinates (regardless of whether this is impossible or unfeasible absent any geocoding methods).

In [151]:
categorical_cols = ['borough', 'contributing_factor_vehicle_1', 'vehicle_type_code1'] # Customize

for col in categorical_cols:
    if col in df.columns and col in df_filtered.columns:
        print(f"\n--- Distribution Comparison for: {col} ---")

        # Calculate proportions, handling potential missing columns gracefully
        try:
            df_dist = df[col].value_counts(normalize=True, dropna=False).rename('Original_df')
        except KeyError:
            print(f"Column '{col}' not found in original df.")
            df_dist = pd.Series(name='Original_df', dtype=float) # Empty series

        try:
            filtered_dist = df_filtered[col].value_counts(normalize=True, dropna=False).rename('Filtered_df')
        except KeyError:
            print(f"Column '{col}' not found in filtered df.")
            filtered_dist = pd.Series(name='Filtered_df', dtype=float) # Empty series

        # Combine for comparison
        comparison_df = pd.concat([df_dist, filtered_dist], axis=1).fillna(0)
        comparison_df['Difference'] = comparison_df['Filtered_df'] - comparison_df['Original_df']

        # Sort by original frequency for better readability (optional)
        comparison_df = comparison_df.sort_values('Original_df', ascending=False)

        print(comparison_df.head(10)) # Show top 10 categories + NaN
    else:
        print(f"\nSkipping column '{col}' as it's not present in both DataFrames.")



--- Distribution Comparison for: borough ---
               Original_df  Filtered_df  Difference
borough                                            
NaN               0.308905     0.242575   -0.066329
BROOKLYN          0.220844     0.242906    0.022061
QUEENS            0.185372     0.203908    0.018537
MANHATTAN         0.153547     0.167410    0.013863
BRONX             0.102346     0.111465    0.009119
STATEN ISLAND     0.028986     0.031736    0.002750

--- Distribution Comparison for: contributing_factor_vehicle_1 ---
                                Original_df  Filtered_df  Difference
contributing_factor_vehicle_1                                       
Unspecified                        0.336523     0.334397   -0.002126
Driver Inattention/Distraction     0.202048     0.205422    0.003374
Failure to Yield Right-of-Way      0.059770     0.061494    0.001723
Following Too Closely              0.053704     0.051879   -0.001825
Backing Unsafely                   0.036638     0.037780

In [152]:
prop_killed_df = (df['number_of_persons_killed'] > 0).mean()
prop_killed_filtered = (df_filtered['number_of_persons_killed'] > 0).mean()

print(f"Proportion of crashes with fatalities (Original): {prop_killed_df:.5f}")
print(f"Proportion of crashes with fatalities (Filtered): {prop_killed_filtered:.5f}")
print(f"Absolute difference: {prop_killed_filtered - prop_killed_df:.5f}")
if prop_killed_df > 0:
     print(f"Relative difference: {(prop_killed_filtered - prop_killed_df) / prop_killed_df:.2%}")

Proportion of crashes with fatalities (Original): 0.00149
Proportion of crashes with fatalities (Filtered): 0.00146
Absolute difference: -0.00003
Relative difference: -1.98%


In [153]:
fatal_crashes_df = (df['number_of_persons_killed'] > 0).sum()
fatal_crashes_filtered = (df_filtered['number_of_persons_killed'] > 0).sum()
fatal_crashes_removed = fatal_crashes_df - fatal_crashes_filtered

print(f"\nTotal fatal crashes (Original): {fatal_crashes_df}")
print(f"Total fatal crashes (Filtered): {fatal_crashes_filtered}")
print(f"Fatal crashes removed by filtering: {fatal_crashes_removed}")
print(f"Proportion of fatal crashes removed: {fatal_crashes_removed / fatal_crashes_df:.2%}")
print(f"Proportion of rows removed: {239932 / 2171175:.2%}")


Total fatal crashes (Original): 3231
Total fatal crashes (Filtered): 2817
Fatal crashes removed by filtering: 414
Proportion of fatal crashes removed: 12.81%
Proportion of rows removed: 11.05%


In [154]:
print("\n--- Value Counts for 'number_of_persons_killed' ---")
df_counts = df['number_of_persons_killed'].value_counts().rename('Original_df')
filtered_counts = df_filtered['number_of_persons_killed'].value_counts().rename('Filtered_df')
counts_comparison = pd.concat([df_counts, filtered_counts], axis=1).fillna(0).astype(int)
print(counts_comparison)


--- Value Counts for 'number_of_persons_killed' ---
                          Original_df  Filtered_df
number_of_persons_killed                          
0.0                           2167913      1928398
1.0                              3128         2728
2.0                                83           70
3.0                                13           13
4.0                                 4            4
5.0                                 2            1
8.0                                 1            1


In [155]:
import numpy as np
from scipy.stats import chi2_contingency

# Create the contingency table from the counts
# Rows: [Fatal, Non-Fatal]
# Columns: [Removed, Kept]
observed_counts = np.array([
    [414, 2817],      # Fatal crashes (Removed, Kept)
    [238586, 1928183] # Non-Fatal crashes (Removed, Kept)
])

# Perform the Chi-Squared test
chi2_stat, p_value, dof, expected_counts = chi2_contingency(observed_counts)

# Output the results
print("--- Chi-Squared Test for Independence ---")
print(f"Contingency Table:\n{observed_counts}\n")
print(f"Chi-Squared Statistic: {chi2_stat:.4f}")
print(f"P-value: {p_value:.4g}") # Use general format for potentially very small p-values
print(f"Degrees of Freedom: {dof}")
# print(f"Expected Counts:\n{expected_counts}\n") # Optional: view expected counts under H0

# Interpretation
alpha = 0.05 # Standard significance level
print(f"\nSignificance Level (alpha): {alpha}")
if p_value < alpha:
    print(f"Result: Reject the null hypothesis (p < {alpha}).")
    print("Conclusion: There is a statistically significant association between crash fatality")
    print("            and the likelihood of being removed by the filter.")
    # Compare proportions to understand direction
    prop_fatal_removed = observed_counts[0, 0] / observed_counts[0, :].sum()
    prop_nonfatal_removed = observed_counts[1, 0] / observed_counts[1, :].sum()
    print(f"  - Proportion of Fatal Crashes Removed: {prop_fatal_removed:.4f} ({prop_fatal_removed:.2%})")
    print(f"  - Proportion of Non-Fatal Crashes Removed: {prop_nonfatal_removed:.4f} ({prop_nonfatal_removed:.2%})")
    if prop_fatal_removed > prop_nonfatal_removed:
        print("  - Fatal crashes were disproportionately *more* likely to be removed.")
    else:
        print("  - Fatal crashes were disproportionately *less* likely to be removed.")
else:
    print(f"Result: Fail to reject the null hypothesis (p >= {alpha}).")
    print("Conclusion: There is no statistically significant evidence to suggest that fatal crashes")
    print("            were removed at a different rate than non-fatal crashes by the filter.")


--- Chi-Squared Test for Independence ---
Contingency Table:
[[    414    2817]
 [ 238586 1928183]]

Chi-Squared Statistic: 10.5087
P-value: 0.001188
Degrees of Freedom: 1

Significance Level (alpha): 0.05
Result: Reject the null hypothesis (p < 0.05).
Conclusion: There is a statistically significant association between crash fatality
            and the likelihood of being removed by the filter.
  - Proportion of Fatal Crashes Removed: 0.1281 (12.81%)
  - Proportion of Non-Fatal Crashes Removed: 0.1101 (11.01%)
  - Fatal crashes were disproportionately *more* likely to be removed.



The Chi-Squared test (assuming it yielded a small p-value based on your numbers) tells us the difference in removal rates (12.81% for fatal vs. ~11.01% for non-fatal) is unlikely to be due to random chance alone, given the large sample size.

However, it doesn't automatically mean the filtered data is "bad" or unusable. We need to assess the magnitude and implication of the difference:

How Big is the Difference?

*Absolute Difference in Removal Rate:* Fatal crashes were removed at a rate that is 1.8 percentage points higher than non-fatal crashes (12.81% - 11.01%).

*Relative Difference in Removal Rate:* The removal rate for fatal crashes was about 16% higher than for non-fatal crashes ((12.81 - 11.01) / 11.01).

*Effect on Remaining Data:* Because fatal crashes were slightly more likely to be removed, the proportion of fatal crashes within df_filtered will be slightly lower than in the original df.

*Original Proportion Fatal:* 3,231 / 2,170,000 ≈ 0.149%

*Filtered Proportion Fatal:* 2,817 / 1,931,000 ≈ 0.146%

The difference in the proportion of fatal crashes in the dataset you'll analyze is only 0.003 percentage points.

In summary, we're going to use the filtered dataset for the rest of this analysis, since it's less concerned with predictors of fatal crashes, and more interested in time series forecasting and neighborhood breakdowns. An additional realization is that most of these locations with unknown lat/lngs are the ones at bridges/tunnels, of which the neighborhood analysis does not apply

In [156]:
df = df_filtered

df.shape

(1931243, 29)

In [157]:
df.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
2,2023-11-01,1:29,BROOKLYN,11230.0,40.621790,-73.970024,"\n, \n(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NaN,...,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN
9,2021-09-11,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"\n, \n(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
10,2021-12-14,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"\n, \n(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
12,2021-12-14,17:05,NaN,NaN,40.709183,-73.956825,"\n, \n(40.709183, -73.956825)",BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4486555,Sedan,Tractor Truck Diesel,NaN,NaN,NaN
13,2021-12-14,8:17,BRONX,10475.0,40.868160,-73.831480,"\n, \n(40.86816, -73.83148)",NaN,NaN,344 BAYCHESTER AVENUE,...,Unspecified,NaN,NaN,NaN,4486660,Sedan,Sedan,NaN,NaN,NaN


In [158]:
# drop the following fields: borough, zip_code, location, on_street_name, cross_street_name, off_street_name, and the last 11 columns
df = df.drop(columns=['borough', 'zip_code', 'location', 'on_street_name', 'cross_street_name', 'off_street_name'] + df.columns[-11:].tolist())

df.head()

,crash_date,crash_time,latitude,longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed
2,2023-11-01,1:29,40.621790,-73.970024,1.0,0.0,0,0,0,0,1,0
9,2021-09-11,9:35,40.667202,-73.866500,0.0,0.0,0,0,0,0,0,0
10,2021-12-14,8:13,40.683304,-73.917274,0.0,0.0,0,0,0,0,0,0
12,2021-12-14,17:05,40.709183,-73.956825,0.0,0.0,0,0,0,0,0,0
13,2021-12-14,8:17,40.868160,-73.831480,2.0,0.0,0,0,0,0,2,0


In [159]:
df.shape

(1931243, 12)

In [160]:
# count of NaN values in each column
df.isna().sum()

crash_date                        0
crash_time                        0
latitude                          0
longitude                         0
number_of_persons_injured        16
number_of_persons_killed         28
number_of_pedestrians_injured     0
number_of_pedestrians_killed      0
number_of_cyclist_injured         0
number_of_cyclist_killed          0
number_of_motorist_injured        0
number_of_motorist_killed         0
dtype: int64

In [161]:
# fill NaN values with 0
df = df.fillna(0)

df.isna().sum()

crash_date                       0
crash_time                       0
latitude                         0
longitude                        0
number_of_persons_injured        0
number_of_persons_killed         0
number_of_pedestrians_injured    0
number_of_pedestrians_killed     0
number_of_cyclist_injured        0
number_of_cyclist_killed         0
number_of_motorist_injured       0
number_of_motorist_killed        0
dtype: int64

In [167]:
df.head()# combine injury tables into a single column, and deaths into

,crash_date,crash_time,latitude,longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,year
2,2023-11-01,1:29,40.621790,-73.970024,1.0,0.0,0,0,0,0,1,0,2023
9,2021-09-11,9:35,40.667202,-73.866500,0.0,0.0,0,0,0,0,0,0,2021
10,2021-12-14,8:13,40.683304,-73.917274,0.0,0.0,0,0,0,0,0,0,2021
12,2021-12-14,17:05,40.709183,-73.956825,0.0,0.0,0,0,0,0,0,0,2021
13,2021-12-14,8:17,40.868160,-73.831480,2.0,0.0,0,0,0,0,2,0,2021


In [168]:
# drop the last 7 columns
df = df.drop(columns=df.columns[-7:].tolist())

df.head()

,crash_date,crash_time,latitude,longitude,number_of_persons_injured,number_of_persons_killed
2,2023-11-01,1:29,40.621790,-73.970024,1.0,0.0
9,2021-09-11,9:35,40.667202,-73.866500,0.0,0.0
10,2021-12-14,8:13,40.683304,-73.917274,0.0,0.0
12,2021-12-14,17:05,40.709183,-73.956825,0.0,0.0
13,2021-12-14,8:17,40.868160,-73.831480,2.0,0.0


In [169]:
df.shape

(1931243, 6)

In [170]:
# write to csv file in /processed

df.to_csv('../data/processed/crashes.csv', index=False)